In [ ]:
# test API endpoints
import requests
import json
from typing import List
import pandas as pd
import polling
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Ping test

In [ ]:
baseurl = "http://localhost:8080/"

# Ping url
endpoint = "ping"

url = baseurl + endpoint
payload={}
files=[]
headers = {}

response = requests.request("GET", url, headers=headers, data=payload, files=files)

print(response.json())

### Get exchanges rates

In [ ]:
# exchange rates
endpoint = "exchange_rates"

url = baseurl + endpoint
payload={}
files=[]
headers = {}

response_exchange_rates = requests.request("GET", url, headers=headers, data=payload, files=files)
exchange_rates = pd.DataFrame(json.loads(response_exchange_rates.json()['exchange_rates']))

### Get company info

In [ ]:
# company_info
endpoint = "company_info"

url = baseurl + endpoint
payload={'companyID': '93614'}
files=[]
headers = {}

response_company_info = requests.request("GET", url, headers=headers, data=payload, files=files)
company_info = pd.DataFrame(json.loads(response_company_info.json()["company_info"]))


In [ ]:
json.loads(response_company_info.json()["company_info"])[0]

### Get SEPA transactions (EUR only)

In [ ]:
# sepa
endpoint = "sepa"

url = baseurl + endpoint
payload={'companyID': '93614'}
files=[]
headers = {}
sepa_df = pd.DataFrame(columns=['id', 'sender', 'beneficiary', 'amount', 'currency', 'time'])
while response_sepa.status_code == 200:
    try:
        response_sepa = requests.request("GET", url, headers=headers, data=payload, files=files, timeout = 5)
    except Exception as e:
        print('endpoint depleted')
        break
    no_rows = response_sepa.text.count("}{") + 1
    response_sepa_text = response_sepa.text
    for row in range(no_rows):
        line_break=response_sepa_text.find("}{")
        if line_break == -1:
            sepa = json.loads(response_sepa_text)['sepa_transactions']
        else:
            sepa = json.loads(response_sepa_text[0:line_break+1])['sepa_transactions']
        sepa_df = pd.concat([sepa_df, pd.DataFrame(json.loads(sepa))])
        response_sepa_text = response_sepa_text[line_break+1:]


In [ ]:
# sepa_df=pd.read_pickle('sepa.pkl')
sepa_df.shape

### Get SWIFT transactions (non EUR)

In [ ]:
# swift
endpoint = "swift"

url = baseurl + endpoint
payload={'companyID': '93614'}
files=[]
headers = {}
swift_df = pd.DataFrame(columns=['id', 'sender', 'beneficiary', 'amount', 'currency', 'time'])
while response_sepa.status_code == 200:
    try:
        response_swift = requests.request("GET", url, headers=headers, data=payload, files=files, timeout = 5)
    except Exception as e:
        print('endpoint depleted')
        break
    no_rows = response_swift.text.count("}{") + 1
    response_swift_text = response_swift.text
    for row in range(no_rows):
        line_break=response_swift_text.find("}{")
        if line_break == -1:
            swift = json.loads(response_swift_text)['swift_transactions']
        else:
            swift = json.loads(response_swift_text[0:line_break+1])['swift_transactions']
        swift_df = pd.concat([swift_df, pd.DataFrame(json.loads(swift))])
        response_swift_text = response_swift_text[line_break+1:]

In [ ]:
swift_df = pd.read_pickle('swift.pkl')
swift_df.shape

### a. Assignment goal

For the Company, we want to find out:

1. Current account(s) Balance.
2. Over-Committed Balance / Debt.
3. Under-Committed Balance / Credit.
4. Countries transacted with.
5. Perform Historical Analysis on the data: Solution must have the ability to view the Account Balance of a company on customizable
   time intervals, i.e., a user can either select a day, week, month or an entire year for analyzing the client.
6. Surprise us.

### 1. Current account balance

In [ ]:
companyId = 10
target_account = company_info[company_info['companyID'] == companyId]['ibans'].iloc[0].replace("{","").replace("}","")
company_name = company_info[company_info['companyID'] == companyId]['name'].iloc[0]
# what if multiple accounts


In [ ]:
# compute balance

account_transactions = pd.DataFrame(columns=['account', 'counterparty', 'date', 'amount', 'drcr', 'currency', 'netAmount'])
# net sepa's
for i, row in sepa_df.query(f"sender == '{target_account}' or beneficiary == '{target_account}'").iterrows():
    
    if row['sender'] == target_account:
        drcr = 'dr'
        dc_sign = -1
        counterparty = row['beneficiary']

    else:
        drcr = 'cr'
        dc_sign = 1
        counterparty = row['sender']
    
    account_transactions = pd.concat(
        [account_transactions,
         pd.DataFrame([
             {'account':target_account,
              'counterparty': counterparty,
              'date': row['time'],
              'amount': row['amount']['Float'],
              'drcr': drcr,
              'currency': 'EUR',
              'netAmountEUR': row['amount']['Float'] * dc_sign
             }
         ]
         )
        ]
    )

# net swifts
for i, row in swift_df.query(f"sender == '{target_account}' or beneficiary == '{target_account}'").iterrows():
    
    if row['sender'] == target_account:
        drcr = 'dr'
        dc_sign = -1
        counterparty = row['beneficiary']
    else:
        drcr = 'cr'
        dc_sign = 1
        counterparty = row['sender']
    
    # calculate EUR amount
    amountEUR = float(row['amount']['Float'] * exchange_rates.query(f"currency == '{row['currency']}'")['eur_rate'])
    
    account_transactions = pd.concat(
        [account_transactions,
         pd.DataFrame([
             {'account':target_account,
              'counterparty': counterparty,
              'date': row['time'],
              'amount': row['amount']['Float'],
              'drcr': drcr,
              'currency': row['currency'],
              'netAmountEUR': amountEUR * dc_sign
             }
         ]
         )
        ]
    )

In [ ]:
# End balance
# What is the balance in Company Info table? should be removed

balance_history = account_transactions.groupby(by=['date']).sum().cumsum()
balance_history['sign'] = balance_history['netAmountEUR'] >= 0
print(f"End balance for {company_name} is {round(float(balance_history.sort_values(by='date').tail(1)['netAmountEUR']),2):,.2f} EUR")


### 2. Over-commited balance

### 3. Under-commited balance

### 4. Balance History

In [ ]:
# Plot balance history
palette = {True:'g', False:'r'} 

sns.set(rc={"figure.figsize":(30, 4)}) #width=3, #height=4
g = sns.scatterplot(data=balance_history, hue="sign", x="date", y="netAmountEUR", palette=palette, legend=None)
set_thicks = g.set_xticks(range(0,len(balance_history),100))

### 4. Accrued interest
Giving the fact that Debit rate = 3% and Credit Rate = 1%, calculate the accrued interest

### 5. Countries transacted with

In [ ]:
account_transactions['counterparty'].str[0:2].unique()

### 6. Balance at date

In [ ]:
def balance_at_date(balance_history, date = '2018-01-01'):
    if date in balance_history.index:
        return balance_history.loc[date, 'netAmountEUR']
    else:
        # find previous balance
        for index, row in balance_history.iterrows():
            if index <= date:
                previous_index = index
                continue
            else:
                return balance_history.loc[previous_index, 'netAmountEUR']
                break

In [ ]:
balance_at_date(balance_history, '2021-10-29')

In [ ]:
# Plot balance history
palette = {True:'g', False:'r'} 

sns.set(rc={"figure.figsize":(30, 4)}) #width=3, #height=4
g = sns.scatterplot(data=balance_history, hue="sign", x="date", y="netAmountEUR", palette=palette, legend=None)
set_thicks = g.set_xticks(range(0,len(balance_history),100), rotation=45)

### 7. Predict Balance
Obs: timerange smoothing might be needed


In [ ]:
bh = balance_history.copy()
bh.index = pd.to_datetime(bh.index, format='%Y-%m-%d')

In [ ]:
train = bh[bh.index < pd.to_datetime("2021-06-01", format='%Y-%m-%d')]
test = bh[bh.index > pd.to_datetime("2021-06-01", format='%Y-%m-%d')]

In [ ]:
y = train['netAmountEUR']

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
ARIMAmodel = ARIMA(y, order = (3, 2, 1))
ARIMAmodel = ARIMAmodel.fit()

y_pred = ARIMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"] 

plt.plot(train['netAmountEUR'], color = "black", label="Train data")
plt.plot(test['netAmountEUR'], color = "red", label="Test data")
plt.plot(y_pred_out, color='Yellow', label = 'ARIMA Predictions')
plt.ylabel('net Balance')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.title("Train/Test split for Balance History")
plt.legend()
plt.show()


# import numpy as np
# from sklearn.metrics import mean_squared_error

# arma_rmse = np.sqrt(mean_squared_error(test["BTC-USD"].values, y_pred_df["Predictions"]))
# print("RMSE: ",arma_rmse)